# Cognyx client debug notebook

This notebook is used to debug the Cognyx client used in cognyx automations.
you will need a .env file with a JWT token to run this notebook. You can get this after logging into the cognyx app.

In the cell below, define the bom_id and view_id you want to use (view_id will default to default view if not provided)

In [1]:
import os

jwt = os.getenv("COGNYX_TOKEN")

bom_id = "5cebf3b2-299a-4253-a365-a2f49d167911"
view_id = None
base_url = "http://localhost:4000"

if not bom_id:
    raise ValueError("bom_id is required")

It will load the bom data and return it.

In [2]:
from cognyx_bom_sdk.cognyx import CognyxClient

cognyx = CognyxClient(base_url=base_url, jwt_token=jwt)

bom_data = await cognyx.load_bom_data(bom_id, view_id)

{'id': '5cebf3b2-299a-4253-a365-a2f49d167911', 'name': 'Complete Kite', 'description': 'Fully assembled kite ready for use outdoors', 'reference': 'KITE-001', 'image': '', 'status': 'draft', 'failure_reason': None, 'created_at': '2025-06-02T10:17:04.000000Z', 'updated_at': '2025-06-04T09:15:07.000000Z', 'auth': {'can': {'view': True, 'update': True, 'delete': True, 'share': True, 'comment': True, 'create': True, 'create_revision': True}}}


When the bom is loaded it will then initialize the BomClient

In [3]:
from cognyx_bom_sdk.client import BomClient


def update_callback(update: dict) -> None:
    """Callback function to handle updates to the BOM."""
    print("Update:", update)


client = BomClient(bom_data=bom_data, update_callback=update_callback)

In [6]:
instance = client.get_bom_instance("child 2")

client.set_instance_attribute(instance.id, "country", "France")

Update: {'type': <BomUpdates.ATTRIBUTE_UPDATE: 'attribute_update'>, 'payload': {'instance_id': '2e301eff-50bc-4a4a-bcad-8ee96c39bf7b', 'attribute_id': '3d610d88-68a1-4b86-95af-9457c96c8804', 'attribute_value': 'France'}}


You can now use the client to interact with the BOM.

In [ ]:
from functools import reduce

from cognyx_bom_sdk.models import BomInstance

mass_attribute = "added mass g"


def get_instance_mass(instance: BomInstance) -> int:
    """Get the mass of a BOM instance.

    Args:
        instance: BOM instance

    Returns:
        The mass of the instance
    """
    instance_mass = client.get_instance_attribute(instance.id, mass_attribute)

    if instance_mass is None:
        return 0

    print(f"instance {instance.name} weighs {instance_mass}g")
    return instance.quantity * instance_mass


reduce(lambda total, instance: total + get_instance_mass(instance), client.list_bom_instances(), 0)

0

In [ ]:
instance = client.get_bom_instance("child 2")

print(instance.id)


NameError: name 'client' is not defined